In [1]:
import sys
sys.path.insert(0, '../src')

from reit_risk_summarizer.services.orchestrator import RiskOrchestrator
import time

## Initialize Orchestrator

Create orchestrator with caching enabled (default).

In [2]:
orchestrator = RiskOrchestrator(
    summarizer_provider="groq",
    summarizer_model="llama-3.3-70b-versatile",
    cache_enabled=True
)

print("Orchestrator initialized")
print(f"Cache enabled: {orchestrator.cache_enabled}")
print(f"Cache size: {orchestrator.cache.size()} entries")

Orchestrator initialized
Cache enabled: True
Cache size: 0 entries


## Test 1: Process AMT (First Run)

Process American Tower - should fetch, extract, and summarize from scratch.

In [3]:
start = time.time()
result = orchestrator.process_reit("AMT")
elapsed = time.time() - start

print(f"\n{'='*80}")
print(f"Ticker: {result.ticker}")
print(f"Company: {result.company_name}")
print(f"Model: {result.model}")
print(f"Processing time: {elapsed:.2f}s")
print(f"\nTop 5 Risks:")
for i, risk in enumerate(result.risks, 1):
    print(f"{i}. {risk}")
    
print(f"\nCache size: {orchestrator.cache.size()} entries")

Groq API error for AMT: Connection error.
Failed to process AMT: Failed to summarize risks for AMT: Groq summarization failed: Connection error.


LLMSummarizationError: Failed to summarize risks for AMT: Groq summarization failed: Connection error.

## Test 2: Process AMT Again (Cache Hit)

Process same REIT - should use cached HTML and risk text, only re-summarize.

In [ ]:
start = time.time()
result2 = orchestrator.process_reit("AMT")
elapsed = time.time() - start

print(f"\nProcessing time (cached): {elapsed:.2f}s")
print(f"Cache size: {orchestrator.cache.size()} entries")
print(f"\nTop 5 Risks:")
for i, risk in enumerate(result2.risks, 1):
    print(f"{i}. {risk}")

## Test 3: Process PLD (Different REIT)

Process Prologis - should fetch and cache new data.

In [ ]:
start = time.time()
result3 = orchestrator.process_reit("PLD")
elapsed = time.time() - start

print(f"\n{'='*80}")
print(f"Ticker: {result3.ticker}")
print(f"Company: {result3.company_name}")
print(f"Processing time: {elapsed:.2f}s")
print(f"\nTop 5 Risks:")
for i, risk in enumerate(result3.risks, 1):
    print(f"{i}. {risk}")
    
print(f"\nCache size: {orchestrator.cache.size()} entries")

## Test 4: Force Refresh

Re-process AMT with `force_refresh=True` - should bypass cache.

In [ ]:
start = time.time()
result4 = orchestrator.process_reit("AMT", force_refresh=True)
elapsed = time.time() - start

print(f"\nProcessing time (force refresh): {elapsed:.2f}s")
print(f"Cache size: {orchestrator.cache.size()} entries")

## Inspect Cache

Check what's currently cached.

In [ ]:
print(f"Cache size: {orchestrator.cache.size()} entries\n")
print("Cached keys:")
for key in sorted(orchestrator.cache.keys()):
    value_length = len(orchestrator.cache.get(key))
    print(f"  {key}: {value_length:,} chars")

## Clear Specific Cache

Clear cache for a specific ticker.

In [ ]:
print(f"Before: {orchestrator.cache.size()} entries")

orchestrator.clear_cache("AMT")

print(f"After clearing AMT: {orchestrator.cache.size()} entries")
print("\nRemaining keys:")
for key in sorted(orchestrator.cache.keys()):
    print(f"  {key}")

## Clear All Cache

Clear entire cache.

In [ ]:
orchestrator.clear_cache()

print(f"Cache size after clearing all: {orchestrator.cache.size()} entries")

## Test Error Handling

Try processing an invalid ticker.

In [ ]:
from reit_risk_summarizer.exceptions import SECFetchError

try:
    result_error = orchestrator.process_reit("INVALID_TICKER_123")
except SECFetchError as e:
    print(f"Expected error: {e}")